https://qiita.com/overlap/items/e7f1077ef8239f454602

In [1]:
import operator, math, random, time
import numpy as np

from deap import algorithms, base, creator, tools, gp

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss

import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# サンプルデータの生成
X, y = make_classification(n_samples=10000, n_features=10, n_informative=5, n_redundant=0, n_repeated=0,
                           n_clusters_per_class=8, random_state=123)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [3]:
# ベースラインスコアの算出
clf = LogisticRegression(penalty="l2", C=1.0)
base_train_auc = np.mean(cross_val_score(clf, X_train, y_train, scoring="roc_auc", cv=5))
clf.fit(X_train, y_train)
base_test_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

In [4]:
# 除算関数の定義
# 左項 / 右項で右項が0の場合1を代入する
def protectedDiv(left, right):
    eps = 1.0e-7
    tmp = np.zeros(len(left))
    tmp[np.abs(right) >= eps] = left[np.abs(right) >= eps] / right[np.abs(right) >= eps]
    tmp[np.abs(right) < eps] = 1.0
    return tmp

In [5]:
# 乱数シード
random.seed(123)

# 適合度を最大化するような木構造を個体として定義
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

In [6]:
# 初期値の計算
# 学習データの5-fold CVのAUCスコアを評価指標の初期値とする
n_features = X_train.shape[1]
clf = LogisticRegression(penalty="l2", C=1.0)
prev_auc = np.mean(cross_val_score(clf, X_train, y_train, scoring="roc_auc", cv=5))

In [7]:
# メインループ
# resultsに特徴量数、学習データのAUCスコア（5-fold CV）、テストデータのAUCスコアを保持する
# exprsに生成された特徴量の表記を保持する
results = []
exprs = []

In [8]:
for i in tqdm(range(10)):
    # 構文木として利用可能な演算の定義
    pset = gp.PrimitiveSet("MAIN", n_features)
    pset.addPrimitive(operator.add, 2)
    pset.addPrimitive(operator.sub, 2)
    pset.addPrimitive(operator.mul, 2)
    pset.addPrimitive(protectedDiv, 2)
    pset.addPrimitive(operator.neg, 1)
    pset.addPrimitive(np.cos, 1)
    pset.addPrimitive(np.sin, 1)
    pset.addPrimitive(np.tan, 1)

    # 関数のデフォルト値の設定
    toolbox = base.Toolbox()
    toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("compile", gp.compile, pset=pset)

    # 評価関数の設定
    # 新しく生成した変数を元の変数に加えて5-fold CVを求める
    def eval_genfeat(individual):
        func = toolbox.compile(expr=individual)
        features_train = [X_train[:,i] for i in range(n_features)]
        new_feat_train = func(*features_train)
        X_train_tmp = np.c_[X_train, new_feat_train]
        return np.mean(cross_val_score(clf, X_train_tmp, y_train, scoring="roc_auc", cv=5)),

    # 評価、選択、交叉、突然変異の設定
    # 選択はサイズ10のトーナメント方式、交叉は1点交叉、突然変異は深さ2のランダム構文木生成と定義
    toolbox.register("evaluate", eval_genfeat)
    toolbox.register("select", tools.selTournament, tournsize=10)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

    # 構文木の制約の設定
    # 交叉や突然変異で深さ5以上の木ができないようにする
    toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))
    toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5)) 

    # 世代ごとの個体とベスト解を保持するクラスの生成
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    # 統計量の表示設定
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    # 進化の実行
    # 交叉確率50%、突然変異確率10%、10世代まで進化
    start_time = time.time()
    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 10, stats=mstats, halloffame=hof, verbose=True)
    end_time = time.time()

    # ベスト解とAUCの保持
    best_expr = hof[0]
    best_auc = mstats.compile(pop)["fitness"]["max"]

    # 5-fold CVのAUCスコアが前ステップのAUCを超えていた場合
    # 生成変数を学習、テストデータに追加し、ベストAUCを更新する
    if prev_auc < best_auc:
        # 生成変数の追加
        func = toolbox.compile(expr=best_expr)
        features_train = [X_train[:,i] for i in range(n_features)]
        features_test = [X_test[:,i] for i in range(n_features)]
        new_feat_train = func(*features_train)
        new_feat_test = func(*features_test)
        X_train = np.c_[X_train, new_feat_train]
        X_test = np.c_[X_test, new_feat_test]

        ### テストAUCの計算（プロット用）
        clf.fit(X_train, y_train)
        train_auc = roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])
        test_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

        # ベストAUCの更新と特徴量数の加算
        prev_auc = best_auc
        n_features += 1

        # 表示と出力用データの保持
        print(n_features, best_auc, train_auc, test_auc, end_time - start_time)
        results.append([n_features, best_auc, train_auc, test_auc])
        exprs.append(best_expr)

        # 変数追加後の特徴量数が30を超えた場合break
        if n_features >= 30:
            break

  0%|          | 0/10 [00:00<?, ?it/s]

   	      	                      fitness                      	              size             
   	      	---------------------------------------------------	-------------------------------
gen	nevals	avg     	max    	min     	std        	avg    	max	min	std    
0  	300   	0.741287	0.75243	0.739287	0.000936722	4.52667	14 	2  	2.84182
1  	164   	0.742463	0.759495	0.738584	0.00261917 	4.42333	14 	1  	2.70385
2  	172   	0.745261	0.768907	0.740169	0.0047919  	4.73   	14 	1  	3.05458
3  	171   	0.749827	0.782676	0.740549	0.00735729 	5.78333	17 	1  	3.96271
4  	156   	0.75783 	0.782676	0.740453	0.0105806  	10.7967	18 	1  	4.05241
5  	168   	0.764185	0.792389	0.740825	0.0141751  	13.09  	22 	3  	2.97689
6  	167   	0.772111	0.793657	0.740837	0.016202   	15.1167	22 	6  	2.57612
7  	171   	0.775656	0.799636	0.740179	0.0174457  	14.83  	24 	4  	2.81681
8  	157   	0.782073	0.801036	0.73717 	0.0180513  	15.55  	22 	1  	2.69459
9  	166   	0.782201	0.801207	0.739749	0.0211176  	16.2   	24 	4  	2.5113

 10%|█         | 1/10 [01:33<13:57, 93.01s/it]

10 	164   	0.786265	0.802428	0.740108	0.0212921  	16.0267	21 	1  	2.47372
11 0.802428302576 0.803795647866 0.797010138382 92.9943859577179
   	      	                         fitness                          	              size             
   	      	----------------------------------------------------------	-------------------------------
gen	nevals	avg     	max     	min     	std       	avg    	max	min	std    
0  	300   	0.802048	0.814404	0.757718	0.00424473	4.28333	14 	2  	2.69377
1  	164   	0.803722	0.814404	0.783152	0.00274161	4.17667	18 	1  	2.61256
2  	164   	0.806679	0.81613 	0.802141	0.0037591 	4.33   	10 	1  	1.86577
3  	161   	0.80984 	0.81613 	0.801938	0.0047213 	4.64333	11 	1  	1.68012
4  	153   	0.81146 	0.81613 	0.802161	0.00515748	5.13667	11 	3  	1.38491
5  	152   	0.81188 	0.81613 	0.802048	0.0051625 	5.62333	9  	1  	1.19225
6  	172   	0.811213	0.81613 	0.801716	0.00557387	5.66   	12 	1  	1.32831
7  	157   	0.811679	0.81613 	0.784686	0.00550372	5.59333	11 	1  	1.24952


 20%|██        | 2/10 [04:34<15:57, 119.67s/it]

...

10 	151   	0.857199	0.85832 	0.854252	0.00147355 	10.3467	19 	1  	2.56382
18 0.858319966421 0.859802014135 0.85161405279 211.95232701301575
   	      	                          fitness                          	             size             
   	      	-----------------------------------------------------------	------------------------------
gen	nevals	avg     	max     	min     	std        	avg    	max	min	std   
0  	300   	0.858292	0.861808	0.857915	0.000264265	4.09667	15 	2  	2.5444
1  	169   	0.858498	0.861808	0.857807	0.000581606	4.06667	14 	1  	2.47701
2  	158   	0.859032	0.861856	0.824596	0.00231578 	4.9    	20 	1  	3.79605
3  	161   	0.860413	0.861881	0.857788	0.00144619 	3.80333	14 	1  	2.33195
4  	165   	0.860713	0.861881	0.829534	0.0023064  	3.85333	12 	1  	1.48048
5  	174   	0.860805	0.86193 	0.856682	0.00151703 	5.44333	11 	1  	1.71273
6  	161   	0.860924	0.862054	0.857997	0.00145993 	6.19   	12 	1  	1.71481
7  	152   	0.861052	0.862054	0.857917	0.00143867 	6.83   	14 	1  	

 90%|█████████ | 9/10 [22:45<02:57, 177.65s/it]

10 	151   	0.861035	0.862286	0.803206	0.0036552  	9.09   	21 	3  	2.74139
19 0.862285550833 0.863790432646 0.853602088539 218.0687222480774
   	      	                         fitness                          	              size             
   	      	----------------------------------------------------------	-------------------------------
gen	nevals	avg     	max     	min     	std       	avg    	max	min	std    
0  	300   	0.862063	0.862926	0.828404	0.00221734	4.10333	14 	2  	2.62158
1  	168   	0.861889	0.863002	0.728203	0.0077349 	4.84667	15 	1  	2.98046
2  	173   	0.862186	0.863058	0.788684	0.00428227	5.06333	18 	1  	3.37234
3  	174   	0.862582	0.863596	0.861955	0.000300234	5.26   	16 	2  	2.86922
4  	158   	0.862719	0.8643  	0.862002	0.000386307	6.41667	12 	1  	2.42413
5  	155   	0.862961	0.864881	0.86181 	0.000561874	7.90333	14 	1  	2.4686 
6  	166   	0.863155	0.864881	0.817466	0.00277137 	8.86   	14 	1  	2.23616
7  	174   	0.863696	0.864884	0.861127	0.00106463 	7.99333	17 	1  	2.

100%|██████████| 10/10 [27:05<00:00, 202.38s/it]

10 	148   	0.864267	0.865027	0.862052	0.00114195 	8.99   	15 	1  	2.01905
20 0.865026864333 0.866722306517 0.855670206291 260.0326159000397


In [9]:
# 結果の出力
print()
print("### Results")
print("Baseline AUC train :", base_train_auc)
print("Baseline AUC test :", base_test_auc)
print("Best AUC train :", results[-1][1])
print("Best AUC test :", results[-1][3])


### Results
Baseline AUC train : 0.741195900486
Baseline AUC test : 0.731194743417
Best AUC train : 0.865026864333
Best AUC test : 0.855670206291


In [10]:
# 結果のプロット
res = np.array(results)
plt.plot(res[:,0], res[:,1],"o-", color="b", label="train(5-fold CV)")
plt.plot(res[:,0], res[:,3],"o-", color="r", label="test")
plt.plot(10, base_train_auc, "d", color="b", label = "train baseline(5-fold CV)")
plt.plot(10, base_test_auc, "d", color="r", label = "test baseline")
plt.xlim(9,31)
plt.grid(which="both")
plt.xlabel('n_features')
plt.ylabel('AUC')
plt.legend(loc="lower right")
plt.savefig("gp_featgen.png")

In [11]:
# 生成した構文木の出力
print()
print("### Generated feature expression")
for expr in exprs:
    print(expr)


### Generated feature expression
mul(sub(add(mul(ARG9, ARG2), cos(neg(ARG6))), neg(ARG6)), sub(ARG7, cos(cos(neg(ARG6)))))
mul(neg(ARG10), mul(ARG9, ARG8))
sub(add(ARG1, ARG8), add(mul(add(ARG2, ARG6), ARG8), cos(protectedDiv(ARG10, ARG3))))
sub(sub(sub(neg(mul(ARG9, ARG12)), cos(ARG9)), cos(ARG9)), mul(ARG2, ARG12))
mul(add(add(ARG6, neg(ARG7)), ARG8), add(sin(ARG6), neg(ARG7)))
sin(cos(add(ARG8, ARG2)))
cos(cos(add(cos(ARG9), sin(sin(ARG2)))))
mul(sub(mul(mul(ARG7, neg(ARG2)), cos(ARG6)), ARG12), ARG16)
mul(neg(sub(sub(sub(ARG14, ARG9), sub(ARG2, ARG16)), ARG7)), ARG8)
mul(sub(ARG14, ARG9), add(add(ARG2, ARG9), add(add(ARG2, ARG9), ARG2)))
